In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from utils.diff import diff

In [2]:
# 01. acc
# 半年频率。遵循Sloan (1996)对应计利润的定义来构建acc：
# acc = [(delta_CA - delta_CASH) - (delta_CL - delta_STD - delta_TP) - Dep]/Total_Assets
# 其中 
# delta表示两个连续周期之间的差
# CA = 流动资产
# CASH = 现金及其等价物
# CL = 流动负债
# STD = 包括在流动负债中的债务
# TP = 应付所得税
# Dep = 折旧及摊销费用

# 02. absacc
# 半年频率。acc的绝对值。

# 60. pctacc
# 半年频率。与acc相同，不同的是分子要除以净收入的绝对值；如果净收入= 0，则净收入设置为0.01作为分母。

# 81. stdacc
# 季度频率（实际计算时改为半年度频率）。16个季度的应计收益的标准差（从第t-16月到第t-1月）。


In [3]:
import pandas as pd
import numpy as np

# 读取资产负债表
df1 = pd.read_csv('季_资产负债表.csv', usecols=['证券代码', '会计期间', '现金及存放中央银行款项', '流动资产合计', '资产总计', '短期借款', '应交税费', '流动负债合计', '查询成功'], low_memory=False)

# 确保证券代码为整数类型
df1['证券代码'] = pd.to_numeric(df1['证券代码'], errors='coerce').fillna(0).astype(int)

# 填充缺失值
df1['现金及存放中央银行款项'][df1['查询成功'].astype('bool')] = df1['现金及存放中央银行款项'][df1['查询成功'].astype('bool')].fillna(0)
df1['短期借款'][df1['查询成功'].astype('bool')] = df1['短期借款'][df1['查询成功'].astype('bool')].fillna(0)

# 读取利润表
df2 = pd.read_csv('季_利润表.csv', usecols=['证券代码', '会计期间', '净利润'])

# 读取现金流量表
df3 = pd.read_csv('季_现金流量表(间接法).csv', usecols=['证券代码', '会计期间', '固定资产折旧、油气资产折耗、生产性生物资产折旧', '无形资产摊销', '长期待摊费用摊销', '查询成功'], low_memory=False)

# 确保证券代码为整数类型
df3['证券代码'] = pd.to_numeric(df3['证券代码'], errors='coerce').fillna(0).astype(int)

# 填充缺失值
df3['固定资产折旧、油气资产折耗、生产性生物资产折旧'][df3['查询成功'].astype('bool')] = df3['固定资产折旧、油气资产折耗、生产性生物资产折旧'][df3['查询成功'].astype('bool')].fillna(0)
df3['无形资产摊销'][df3['查询成功'].astype('bool')] = df3['无形资产摊销'][df3['查询成功'].astype('bool')].fillna(0)
df3['长期待摊费用摊销'][df3['查询成功'].astype('bool')] = df3['长期待摊费用摊销'][df3['查询成功'].astype('bool')].fillna(0)

# 合并数据
df4 = pd.merge(pd.merge(df1, df2, on=['证券代码', '会计期间'], how='left'), df3, on=['证券代码', '会计期间'], how='left')

# 处理会计期间
df4['半年'] = df4['会计期间'].apply(lambda x: True if x[5:7] in ['06', '12'] else False)
df5 = df4[df4['半年']].reset_index(drop=True)

# 计算差异
# 假设 diff 函数已经定义
df5 = diff(df5, col=['净利润', '固定资产折旧、油气资产折耗、生产性生物资产折旧', '无形资产摊销', '长期待摊费用摊销'], freq='半年')

# 计算折旧摊销
df5['折旧摊销'] = df5['固定资产折旧、油气资产折耗、生产性生物资产折旧'] + df5['无形资产摊销'] + df5['长期待摊费用摊销']

# 计算变化量
df5['delta_流动资产合计'] = df5['流动资产合计'] - df5.groupby('证券代码')['流动资产合计'].shift(1)
df5['delta_现金及存放中央银行款项'] = df5['现金及存放中央银行款项'] - df5.groupby('证券代码')['现金及存放中央银行款项'].shift(1)
df5['delta_流动负债合计'] = df5['流动负债合计'] - df5.groupby('证券代码')['流动负债合计'].shift(1)
df5['delta_短期借款'] = df5['短期借款'] - df5.groupby('证券代码')['短期借款'].shift(1)
df5['delta_应交税费'] = df5['应交税费'] - df5.groupby('证券代码')['应交税费'].shift(1)

# 计算 acc 相关指标
df5['acc'] = ((df5['delta_流动资产合计'] - df5['delta_现金及存放中央银行款项']) - 
              (df5['delta_流动负债合计'] - df5['delta_短期借款'] - df5['delta_应交税费']) - 
              df5['折旧摊销']) / df5['资产总计'].replace(0, np.nan)

df5['absacc'] = abs(df5['acc'])
df5['pctacc'] = df5['acc'] * df5['资产总计'] / df5['净利润']
df5['stdacc'] = df5.groupby('证券代码')['acc'].rolling(3).std().reset_index(drop=True)

# 选择最终列并导出
df6 = df5[['证券代码', '会计期间', 'acc', 'absacc', 'pctacc', 'stdacc']]
df6.to_csv('半年_1_2_60_81.csv', index=False, encoding='utf-8-sig')
df6

,证券代码,会计期间,acc,absacc,pctacc,stdacc
0,1,2000-06-30,NaN,NaN,NaN,NaN
1,1,2000-12-31,NaN,NaN,NaN,NaN
2,1,2001-06-30,NaN,NaN,NaN,NaN
3,1,2001-12-31,NaN,NaN,NaN,NaN
4,1,2002-06-30,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
282676,920118,2022-06-30,NaN,NaN,NaN,NaN
282677,920118,2022-12-31,NaN,NaN,NaN,NaN
282678,920118,2023-06-30,NaN,NaN,NaN,NaN
282679,920118,2023-12-31,NaN,NaN,NaN,NaN
